<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generate Readme for Awesome Notebooks

In [1]:
import os
import naas_drivers
import urllib.parse

import copy
import nbformat

from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)

## Variables

In [2]:
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/jupyter-naas/awesome-notebooks/tree/master'
github_download_url = 'https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/'
naas_download_url='https://app.naas.ai/user-redirect/naas/downloader?url='
naas_logo='https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo=data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz4KPHN2ZyB3aWR0aD0iMTAyNHB4IiBoZWlnaHQ9IjEwMjRweCIgdmlld0JveD0iMCAwIDEwMjQgMTAyNCIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIj4KIDwhLS0gR2VuZXJhdGVkIGJ5IFBpeGVsbWF0b3IgUHJvIDIuMC41IC0tPgogPGRlZnM+CiAgPHRleHQgaWQ9InN0cmluZyIgdHJhbnNmb3JtPSJtYXRyaXgoMS4wIDAuMCAwLjAgMS4wIDIyOC4wIDU0LjUpIiBmb250LWZhbWlseT0iQ29tZm9ydGFhLVJlZ3VsYXIsIENvbWZvcnRhYSIgZm9udC1zaXplPSI4MDAiIHRleHQtZGVjb3JhdGlvbj0ibm9uZSIgZmlsbD0iI2ZmZmZmZiIgeD0iMS4xOTk5OTk5OTk5OTk5ODg2IiB5PSI3MDUuMCI+bjwvdGV4dD4KIDwvZGVmcz4KIDx1c2UgaWQ9Im4iIHhsaW5rOmhyZWY9IiNzdHJpbmciLz4KPC9zdmc+Cg=='

## Get files list

In [3]:
total = []
for root, directories, files in os.walk(current_file, topdown=False):
    total.append({"root": root, "directories":directories, "files":files})
total.sort(key=lambda x: x.get('root'))

## Set 'Naas Download' link on notebook

In [4]:
def get_title(folder_nice, file_nice, download_link):
    return f"""# {folder_nice} - {file_nice}\n<a href="{download_link}" target="_parent">\n<img src="{naas_logo}"/>\n</a>"""

def set_notebook_title(notebook_path, title_source):
    header_found = False
    count = 0
    nb = load_notebook_node(notebook_path)
    nb = copy.deepcopy(nb)
    for cell in nb.cells:
        source = cell.source
        if cell.cell_type == "code":
            nb.cells[count].outputs = []
        if  header_found is False and cell.cell_type == "markdown" and len(source) > 2 and source[0] == '#' and source[1] == ' ':
            nb.cells[count].source = title_source
            header_found = True
        count += 1
    write_ipynb(nb, notebook_path)


## Convert filepath in Markdown text

In [5]:
def get_file_md(folder_nice, folder_url, files, title_sep="##", subtitle_sep="*"):
    md = ""
    if (len(files) > 0):
        md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
    #                 print(file)

            if file.endswith(notebook_ext):
                file_url = urllib.parse.quote(file)
                file_nice = file.replace('_', ' ')
                file_nice = file_nice.replace(notebook_ext, '')
                file_nice = file_nice.replace(folder_nice, '')
                file_nice = file_nice.strip()
                if (file_nice != ""):
                    file_nice = file_nice[0].capitalize() + file_nice[1:]
                path = urllib.parse.unquote(f"{folder_url}/{file_url}")
                title = get_title(folder_nice, file_nice, f"{naas_download_url}{github_download_url}{folder_url}/{file_url}")
                set_notebook_title(path, title)
                md += f"{subtitle_sep} [{file_nice}]({github_url}/{folder_url}/{file_url})\n"
    return md

## Create list of all notebooks

In [6]:
generated_list = ""

for cur in total:
    root = cur.get('root')
    md_round = ""
    directories = cur.get('directories') 
    files = cur.get('files')
    files = sorted(files)
    if ('.git' not in root and '.ipynb_checkpoints' not in root and '.' != root):
        folder_nice = root.replace('./', '')
        folder_url = urllib.parse.quote(folder_nice)
        if ('/' not in folder_nice):
            md_round += get_file_md(folder_nice, folder_url, files)
        elif ('/' in folder_nice):
            folder_url = urllib.parse.quote(folder_nice)
            subfolder_nice = folder_nice.split('/')[1].replace('_', ' ').replace(folder_nice, '').strip()
            md_round += get_file_md(subfolder_nice, folder_url, files, "\t###", "\t-")
    elif ('.ipynb_checkpoints' in root):
#         print(root, files)
        for file in files:
            try:
                os.remove(os.path.join(root, file))
            except:
                pass
        try:
            os.rmdir(root)
        except:
            pass
#     print(md_round)
    generated_list += md_round   

## Preview the generated list

In [7]:
naas_drivers.markdown.display(generated_list)

In [8]:
template = open(readme_template).read()

template = template.replace(replace_var, generated_list)

f  = open(readme, "w+")
f.write(template)
f.close()